


so i create a universe object with mda (using the pdb and xtc files)

now i should use py3dmol and generate movie out of the samples

then i should do rmsd analysis on the samples

then i have to look at the different conformations and come up with conjectures and device strategies to test those conjectures



the first step is pretty straightforward

the second step helps me approach the 3rd and 4th steps with better understanding
this step i can achieve by rendering them all in sequence; there seem to be straightforward instructions on how to render the view using a pdb file; *how do i do this for xtc files*

rmsd analysis should be straightforward once i get a hang 


python -m bioemu.sample --sequence MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG --num_samples 200 --output_dir ./test-bioemu-ubq



In [1]:
import MDAnalysis as mda 

u = mda.Universe("topology.pdb", "samples.xtc")

print(u)

def visualize_bioemu_samples_in_mda_universe(u, sel_string="not ((resname WAT) or (resname HOH))", style={"cartoon": {'color': 'spectrum'}}):
    # The number of frames in the simulation
    number_frames_analysis = len(u.trajectory)
    if number_frames_analysis > 10:
        stride_animation = number_frames_analysis/1 #100 KC
    else:
        stride_animation = 1


    class Atom(dict):
        def __init__(self, line):
            self["type"] = line[0:6].strip()
            self["idx"] = line[6:11].strip()
            self["name"] = line[12:16].strip()
            self["resname"] = line[17:20].strip()
            self["resid"] = line[22:26].strip()   
            self["x"] = line[30:38].strip()
            self["y"] = line[38:46].strip()
            self["z"] = line[46:54].strip()
            #self["occupancy"] = line[54:60].strip()
            #self["temp factor"] = line[60:66].strip()
            self["sym"] = line[76:78].strip()

        def __str__(self):
            record = ""
            record += f"{self['type']:<6}"
            record += f"{self['idx']:<5}"
            record += f"{self['name']:<4}"
            record += f"{self['resname']:<4}"
            record += f"{self['resid']:<5}"
            record += f"{self['x']:>8}"
            record += f"{self['y']:>8}"
            record += f"{self['z']:>8}"
            #record += f"{self['occupancy']:>6}"
            #record += f"{self['temp factor']:>6}"
            record += f"{self['sym']:>2}"
            return record+"\n"
            
    class Molecule(list):
        def __init__(self, file):
            for line in file:
                if line.startswith("ATOM") or line.startswith("HETATM"):
                    self.append(Atom(line))
                
        def __str__(self):
            model = ""
            for atom in self:
                model += str(atom)
            return model

    # write some files with just the protein
    protein = u.select_atoms(sel_string) 
    i = 0
    for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:  # TODO: why unused variable ts
        if i > -1:
            with mda.Writer(f"sample_{i}.pdb", protein.n_atoms) as W:
                W.write(protein)
        i = i + 1


    molecules = []
    for i in range(len(u.trajectory)):
        with open(f"sample_{i}.pdb") as file:
            molecules.append(Molecule(file))

            
    print(molecules)
    # molecules seems to be fine; it must be the models
    models = ""
    for model in range(len(molecules)):
        models += "MODEL " + str(model) + "\n"
        for j, mol in enumerate(molecules[model]):
            models += str(mol)
        models += "ENDMDL\n"
    
    import py3Dmol
    view = py3Dmol.view(width=800, height=600)
    view.addModelsAsFrames(models)
    print(models)
    for i, at in enumerate(molecules[0]):
         view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", style))
    
    view.zoomTo()
    view.animate({'loop': "forward", 'reps': 0})#, 'duration': 1000})
    return view



<Universe with 374 atoms>


In [2]:
# Call the function to visualize the samples
view = visualize_bioemu_samples_in_mda_universe(u)
view.show()


/home/uni/capstone/.venv/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/uni/capstone/.venv/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


FileNotFoundError: [Errno 2] No such file or directory: 'sample_1.pdb'

In [ ]:
import visualize

import MDAnalysis as mda 

u = mda.Universe("topology.pdb", "samples.xtc")
view = visualize.visualize_mda_universe(u)
view.show()


In [ ]:
# takes the path of PDB file using os module to join the directory in a platform-independent way
import os
DATA = os.path.join('/home', 'uni', 'capstone', 'EROS_Ub/')
print(DATA)

#creates a universe object from the PDB file alone

import MDAnalysis as mda
mda_universe = mda.Universe(os.path.join(DATA, "2k39.pdb")) 
# why is this object creation taking only topology file and no trajectory file
print(mda_universe)

# This function does many things:
# 1. It takes a mdanalysis universe object
# 2. It takes an optional argument selection string to select atoms to visualize
# 3. It also takes an optional argument style to set the visualization style; not imp at the moment

# 5. Cleans up previously stored PDBs storing frames
# 6. It creates object blueprints for atoms and molecules in the PDB files to be read;
#   there are two use cases for the atom and molecule objects: to print the object, taken care of by __str__ method
#   and to write the object to a PDB file, taken care of by __str__ method;
#   the __init__ method takes care of reading the PDB file and creating the object

# 7. write frames to files
# 8. list of molecule objects is created and a string of all molecules called models is created

def visualize_mda_universe(u, \
    sel_string='not ((resname WAT) or (resname HOH))', 
    style={"cartoon": {'color': 'spectrum'}}):
  
  """
  Inputs: 
  u : mdanalysis universe
  sel_string : mdanalysis selection string for visible atoms
  style : py3Dmol style
  """


  # The number of frames in the simulation
  number_frames_analysis = len(u.trajectory)
  if number_frames_analysis > 10:
    stride_animation = number_frames_analysis/1 #100 KC
  else:
    stride_animation = 1

  # Deleting previously stored frames as PDBs and removing warnings
  import warnings
  warnings.filterwarnings('ignore')
  !rm [0-9]?.pdb 2> /dev/null
  

  # TODO: do i need to assert that the line has no more than 80 characters?
    # Helper classes to read and get PDB fields
  class Atom(dict):
    def __init__(self, line):
      self["type"] = line[0:6].strip()
      self["idx"] = line[6:11].strip()
      self["name"] = line[12:16].strip()      # TODO : what is this
      self["resname"] = line[17:20].strip()   # TODO: there is something between residue name and residue id what is it, in the file
      self["resid"] = int(int(line[22:26]))
      #print(type(self["resid"]))
      #print(self["resid"])  
      self["x"] = float(line[30:38])
      self["y"] = float(line[38:46])
      self["z"] = float(line[46:54])          
      # there are two thing after x y and z coordinates, and before symbol of atom; 
      # The second one is B-factor or temperature factor (as explained in one of the papers in the literature)
      # The first one is a quantity describing the amount of time spent 
      # in that position described by x y and z; it's called occupancy 
      # # Occupancy comes in handy to note if the molecule is flexible in that region
      self["sym"] = line[76:78].strip()

    def __str__(self):
      line = list(" " * 80)                   # initializes a line with 80 characters
      line[0:6] = self["type"].ljust(6)       
      line[6:11] = self["idx"].ljust(5)
      line[12:16] = self["name"].ljust(4)
      line[17:20] = self["resname"].ljust(3)
      line[22:26] = str(self["resid"]).ljust(4)
      line[30:38] = str(self["x"]).rjust(8)
      line[38:46] = str(self["y"]).rjust(8)
      line[46:54] = str(self["z"]).rjust(8)
      line[76:78] = self["sym"].rjust(2)
      return "".join(line) + "\n"             # returns the line after updating the characters to reflect atom attributes
          
  class Molecule(list):
    def __init__(self, file):                 # collects all lines labelled ATOM
      for line in file:
        if "ATOM  " in line or "HETATM" in line:      # TODO: what is HETATM; heterogeneous atom is an atom in a small molecule
          self.append(Atom(line))
              
      def __str__(self):
        outstr = ""
        for at in self:
          outstr += str(at)
        return outstr

  # Write out frames for animation
  protein = u.select_atoms(sel_string) 
  i = 0
  for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:  # TODO: why unused variable ts
      if i > -1:
          with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:  
              W.write(protein)
      i = i + 1

  # TODO: why write to files at all if all you are going to do with the files is to  put them all in the string models later


  # Load frames as molecules (py3Dmol let us visualize a single "molecule" per frame)
  molecules = []
  for i in range(int(len(u.trajectory)/int(stride_animation))):
      with open('' + str(i) + '.pdb') as ifile:
          molecules.append(Molecule(ifile))
  print(molecules)
  models = ""
  for i in range(len(molecules)):
    models += "MODEL " + str(i) + "\n"
    for j,mol in enumerate(molecules[i]):  # TODO: shouldn't the variable here be atom instead of mol?
      # i think it should be atom;
      # TODO: figure out a way to get rid of the useless j variable
      models += str(mol)
    models += "ENDMDL\n"


  # Animation
  import py3Dmol
  view = py3Dmol.view(width=800, height=600)
  view.addModelsAsFrames(models)
  print(models)
  for i, at in enumerate(molecules[0]):
      view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", style))

  view.zoomTo()
  view.animate({'loop': "forward", 'reps': 0}) # TODO: explain this line
  return view

#creating a view object of py#Dmol module to view the animation

view = visualize_mda_universe(mda_universe)
view.show()